<a href="https://colab.research.google.com/github/lucasbrancalhao/mercadofinanceiro/blob/main/Correla%C3%A7%C3%A3o_entre_a%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['WEGE3.SA', 'PETR4.SA', 'MGLU3.SA', 'ITSA4.SA']
# WEG, Petrobras, Magalu e Itausa

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                          Open       High        Low      Close  Adj Close  \
Ticker   Date                                                                
WEGE3.SA 2023-03-06  38.990002  39.590000  38.700001  39.430000  39.377354   
         2023-03-07  39.299999  39.770000  39.139999  39.590000  39.537140   
         2023-03-08  39.599998  40.320000  39.419998  40.160000  40.106377   
         2023-03-09  40.000000  40.299999  39.860001  39.860001  39.806778   
         2023-03-10  39.820000  39.990002  39.320000  39.540001  39.487206   

                      Volume  
Ticker   Date                 
WEGE3.SA 2023-03-06  5594500  
         2023-03-07  4956200  
         2023-03-08  7378100  
         2023-03-09  5

In [ ]:
df = df.reset_index()
print(df.head())

     Ticker       Date       Open       High        Low      Close  Adj Close  \
0  WEGE3.SA 2023-03-06  38.990002  39.590000  38.700001  39.430000  39.377354   
1  WEGE3.SA 2023-03-07  39.299999  39.770000  39.139999  39.590000  39.537140   
2  WEGE3.SA 2023-03-08  39.599998  40.320000  39.419998  40.160000  40.106377   
3  WEGE3.SA 2023-03-09  40.000000  40.299999  39.860001  39.860001  39.806778   
4  WEGE3.SA 2023-03-10  39.820000  39.990002  39.320000  39.540001  39.487206   

    Volume  
0  5594500  
1  4956200  
2  7378100  
3  5384100  
4  4632900  


In [ ]:
import plotly.express as px
fig = px.line(df, x='Date', 
              y='Close', 
              color='Ticker', 
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [ ]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()


In [ ]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for ITSA4.SA
      MA10    MA20
186    NaN     NaN
187    NaN     NaN
188    NaN     NaN
189    NaN     NaN
190    NaN     NaN
..     ...     ...
243  8.957  8.8535
244  8.947  8.8785
245  8.924  8.8990
246  8.914  8.9195
247  8.931  8.9375

[62 rows x 2 columns]
Moving Averages for MGLU3.SA
      MA10    MA20
124    NaN     NaN
125    NaN     NaN
126    NaN     NaN
127    NaN     NaN
128    NaN     NaN
..     ...     ...
181  3.638  3.7840
182  3.673  3.8085
183  3.717  3.8315
184  3.769  3.8560
185  3.811  3.8685

[62 rows x 2 columns]
Moving Averages for PETR4.SA
       MA10     MA20
62      NaN      NaN
63      NaN      NaN
64      NaN      NaN
65      NaN      NaN
66      NaN      NaN
..      ...      ...
119  26.200  25.2695
120  26.209  25.4520
121  26.255  25.6250
122  26.370  25.8220
123  26.511  25.9810

[62 rows x 2 columns]
Moving Averages for WEGE3.SA
      MA10     MA20
0      NaN      NaN
1      NaN      NaN
2      NaN      NaN
3      NaN      NaN
4      

In [ ]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

In [ ]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

In [ ]:
# create a DataFrame with the stock prices of Apple and Microsoft
petr = df.loc[df['Ticker'] == 'PETR4.SA', ['Date', 'Close']].rename(columns={'Close': 'PETR4.SA'})
itsa = df.loc[df['Ticker'] == 'MGLU3.SA', ['Date', 'Close']].rename(columns={'Close': 'MGLU3.SA'})
df_corr = pd.merge(petr, itsa, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='PETR4.SA', y='MGLU3.SA', 
                 trendline='ols', 
                 title='Correlation between PETR4 and ITSA4')
fig.show()